#### DataGeneration ver1 이용한 training data / test data 생성 후,
#### 생성된 training data / test data 에 포함된 target distribution 확인 

In [1]:
import numpy as np
import random
from datetime import datetime

# 수치미분 함수

def numerical_derivative(f, x):
    delta_x = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    
    while not it.finished:
        idx = it.multi_index        
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + delta_x
        fx1 = f(x) # f(x+delta_x)
        
        x[idx] = tmp_val - delta_x 
        fx2 = f(x) # f(x-delta_x)
        grad[idx] = (fx1 - fx2) / (2*delta_x)
        
        x[idx] = tmp_val 
        it.iternext()   
        
    return grad

# sigmoid 함수

def sigmoid(x):
    return 1 / (1+np.exp(-x))

#### Wine 클래스 정의

In [2]:
# Wine Class

class Wine:
    
    # 생성자
    # xdata, tdata => numpy.array(...)
    def __init__(self, input_nodes, hidden_nodes, output_nodes, learning_rate):
        
        # 2층 hidden layer unit 
        # 가중치 W, 바이어스 b 초기화
        self.W2 = np.random.rand(input_nodes, hidden_nodes)  
        self.b2 = np.random.rand(hidden_nodes)
        
        # 3층 output layer unit : 1 개 
        self.W3 = np.random.rand(hidden_nodes,output_nodes)
        self.b3 = np.random.rand(output_nodes)
                        
        # 학습률 learning rate 초기화
        self.learning_rate = learning_rate
        
        print("Wine object is created !!!")
        
    # 손실함수
    def feed_forward(self):
        
        delta = 1e-7    # log 무한대 발산 방지
    
        z2 = np.dot(self.input_data, self.W2) + self.b2
        a2 = sigmoid(z2)
        
        z3 = np.dot(a2, self.W3) + self.b3
        y = a3 = sigmoid(z3)
    
        # cross-entropy 
        return  -np.sum( self.target_data*np.log(y + delta) + (1-self.target_data)*np.log((1 - y)+delta ) )
    
    # obtain W and b
    def get_W_b(self):
        
        return self.W2,  self.b2, self.W3, self.b3
    
    # 손실 값 계산
    def loss_val(self):
        
        delta = 1e-7    # log 무한대 발산 방지
    
        z2 = np.dot(self.input_data, self.W2) + self.b2
        a2 = sigmoid(z2)
        
        z3 = np.dot(a2, self.W3) + self.b3
        y = a3 = sigmoid(z3)
    
        # cross-entropy 
        return  -np.sum( self.target_data*np.log(y + delta) + (1-self.target_data)*np.log((1 - y)+delta ) )
    
    # query, 즉 미래 값 예측 함수
    def predict(self, input_data):    
        
        z2 = np.dot(input_data, self.W2) + self.b2
        a2 = sigmoid(z2)
        
        z3 = np.dot(a2, self.W3) + self.b3
        y = a3 = sigmoid(z3)
    
        if y >= 0.5:
            result = 1  # True
        else:
            result = 0  # False
    
        return y, result

    
    def accuracy(self, input_data, target_data):
        
        matched_list = []
        not_matched_list = []
        
        # list which contains (index, label, prediction) value
        index_label_prediction_list = []
        
        # temp list which contains label and prediction in sequence
        temp_list = []
        
        for index in range(len(input_data)):
            
            (real_val, logical_val) = self.predict(input_data[index])
            
            if logical_val == target_data[index]:
                matched_list.append(index)
            else:
                not_matched_list.append(index)
                
                temp_list.append(index)
                temp_list.append(target_data[index])
                temp_list.append(logical_val)
                
                index_label_prediction_list.append(temp_list)
                
                temp_list = []
                
                
        accuracy_result = len(matched_list) / len(input_data)
        
        print("Accuracy => ", accuracy_result)
        
        return matched_list, not_matched_list, index_label_prediction_list
    
        
    # 수치미분을 이용하여 손실함수가 최소가 될때 까지 학습하는 함수
    def train(self, input_data, target_data):
        
        self.input_data = input_data
        self.target_data = target_data
        
        f = lambda x : self.feed_forward()
        
        self.W2 -= self.learning_rate * numerical_derivative(f, self.W2)
    
        self.b2 -= self.learning_rate * numerical_derivative(f, self.b2)
        
        self.W3 -= self.learning_rate * numerical_derivative(f, self.W3)
    
        self.b3 -= self.learning_rate * numerical_derivative(f, self.b3)

#### DataGeneration class 이용하여 training_data,  test_data 분리

In [3]:
class DataGeneration:
    
    # target_position = 0 (첫번째열이 정답데이터), target_position=-1 (마지막열이 정답데이터)
    def __init__(self, name, file_path, seperation_rate, target_position=-1):
        
        self.name = name
        
        self.file_path = file_path
        
        self.seperation_rate = seperation_rate
        
        if (target_position == -1  or  target_position == 0):      
            self.target_position = target_position
        
        else:
            err_str = 'target_position must be -1 or 0'            
            raise Exception(err_str)    
            
    
    # print data target distribution 
    # str_of_kind : 'original data' or  'training data'  or  'test data'
    def print_target_distribution(self, data, str_of_kind='original data'):
        
        print('=======================================================================================================')
        
        target_data = data[ :, self.target_position ]
        
        # numpy.unique() 사용하여 loaded data target 분포 확인
        unique, counts = np.unique(target_data, return_counts=True)

        print('[DataGeneration]  ', str_of_kind, ' target value = ', dict(zip(unique, counts)).items())

        num_zeros = dict(zip(unique, counts))[0.0]  # key 0.0 에 대한 value 값 count 리턴
        num_ones = dict(zip(unique, counts))[1.0]  # key 1.0 에 대한 value 값 count 리턴

        print('[DataGeneration]  ', str_of_kind, ' zeros numbers = ', num_zeros, ', ratio = ', 100 * num_zeros / (data.shape[0]), ' %')
        print('[DataGeneration]  ', str_of_kind, ' ones numbers = ', num_ones, ', ratio = ', 100 * num_ones / (data.shape[0]), '%') 
    
        print('=======================================================================================================')
        
        
    # shuffle 기능을 이용하여 training_data / test_data 생성
    def generate(self):
    
        # 데이터 불러오기, 파일이 없는 경우 exception 발생

        try:
            loaded_data = np.loadtxt(self.file_path, delimiter=',', dtype=np.float32)
            
        except Exception as err:
            print('[DataGeneration::generate()]  ', str(err))
            raise Exception(str(err))

        print("[DataGeneration]  loaded_data.shape = ", loaded_data.shape)
            
        # print the target distribution of original data 
        
        self.print_target_distribution(loaded_data, 'original data')
        
        
        # random.shuffle() 이용한 데이터 인덱스 분리 및 트레이닝/테스트 데이터 생성
        
        # 임시 저장 리스트
        training_data_list = []
        test_data_list = []

        # 분리비율에 맞게 테스트데이터로 분리
        total_data_num = len(loaded_data)
        test_data_num = int(len(loaded_data) * self.seperation_rate)

        #print("[DataGeneration]  total_data_num = ", total_data_num, ", test_data_num = ", test_data_num)

        # 전체 데이터 인덱스를 가지고 있는 리스트 생성
        total_data_index_list = [ index for index in range(total_data_num) ]

        # random.shuffle 을 이용하여 인덱스 리스트 생성
        random.shuffle(total_data_index_list)  # 전체 인덱스가 랜덤하게 섞여진 리스트로 변형된다

        # test data 를 위한 인덱스는 total_data_index_list 로뷰터 앞에서 분리비율(seperation_rate)의 데이터 인덱스
        test_data_index_list = total_data_index_list[ 0:test_data_num ]

        #print("[DataGeneration]  length of test_data_index_list = ", len(test_data_index_list))

        # training data 를 위한 인덱스는 total_data_index_list 에서 test data 인덱스를 제외한 나머지 부분
        training_data_index_list = total_data_index_list[ test_data_num: ]

        #print("[DataGeneration]  length of training_data_index_list = ", len(training_data_index_list))

        # training data 구성
        for training_data_index in training_data_index_list:
    
            training_data_list.append(loaded_data[training_data_index])

        # test data 구성
        for test_data_index in test_data_index_list:
    
            test_data_list.append(loaded_data[test_data_index])

        # generate training data from training_data_list using np.arrya(...)
        training_data = np.array(training_data_list)

        # generate test data from test_data_list using np.arrya(...)
        test_data = np.array(test_data_list)

        # verification shape
        #print("[DataGeneration]  training_data.shape = ", training_data.shape)
        #print("[DataGeneration]  test_data.shape = ", test_data.shape)

        # print target distribution of generated data 
        
        self.print_target_distribution(training_data, 'training data')
        
        self.print_target_distribution(test_data, 'test data')
        
        
        # save training & test data (.csv)
        training_data_save_path = './' + self.name + '_training_data.csv'
        test_data_save_path = './' + self.name + '_test_data.csv'
        
        # 저장공간이 없거나 파일 write 실패시 exception 발생
        try:
            np.savetxt(training_data_save_path, training_data, delimiter=',')
            np.savetxt(test_data_save_path, test_data, delimiter=',')
            
        except Exception as err:
            print('[DataGeneration::generate()]  ', str(err))
            raise Exception(str(err))
        
        return training_data, test_data

#### DataGeneration 객체 생성.
#### 원본데이터 및 생성된 training data / test data 에 대한 정답(target) 분포 확인 기능 추가 

In [4]:
# DataGeneration 객체 생성.
# 생성된 training data / test data 에 대한 정답(target) 분포 확인

seperation_rate = 0.3 # 분리비율
target_position = -1

try:    
    # DataGeneration 이용한 training data / test data 분리
    
    data_obj = DataGeneration('Wine', './(191109)wine.csv', seperation_rate, target_position)

    (training_data, test_data) = data_obj.generate()
    
except Exception as err:
    
    print('Exception Occur !!')
    print(str(err))

[DataGeneration]  loaded_data.shape =  (6497, 13)
[DataGeneration]   original data  target value =  dict_items([(0.0, 4898), (1.0, 1599)])
[DataGeneration]   original data  zeros numbers =  4898 , ratio =  75.38864091118978  %
[DataGeneration]   original data  ones numbers =  1599 , ratio =  24.61135908881022 %
[DataGeneration]   training data  target value =  dict_items([(0.0, 3432), (1.0, 1116)])
[DataGeneration]   training data  zeros numbers =  3432 , ratio =  75.46174142480211  %
[DataGeneration]   training data  ones numbers =  1116 , ratio =  24.53825857519789 %
[DataGeneration]   test data  target value =  dict_items([(0.0, 1466), (1.0, 483)])
[DataGeneration]   test data  zeros numbers =  1466 , ratio =  75.21806054386865  %
[DataGeneration]   test data  ones numbers =  483 , ratio =  24.78193945613135 %


#### Hyper-Parameter 설정 및 train 실행

In [5]:
#hyper-parameter
i_nodes = training_data.shape[1] - 1    # input nodes 개수
h1_nodes = 15  # hidden nodes 개수
o_nodes = 1    # output nodes 개수
lr = 1e-5      # learning rate
epochs = 20   # 반복횟수

# Wine 객체 생성
obj = Wine(i_nodes, h1_nodes, o_nodes, lr)

print("Neural Network Learning using Numerical Derivative...")

start_time = datetime.now()

for step in range(epochs):
    
    for index in range(len(training_data)):
        
        input_data = training_data[index, 0:-1]
        target_data = training_data[index, [-1]]
        
        obj.train(input_data, target_data)
        
    if (step % 2 == 0):
        print("epochs = ", step, "loss value = ", obj.loss_val())

end_time = datetime.now()
        
print("")
print("Elapsed Time => ", end_time - start_time)

Wine object is created !!!
Neural Network Learning using Numerical Derivative...
epochs =  0 loss value =  6.938813362363636
epochs =  2 loss value =  5.845202373258166
epochs =  4 loss value =  4.760329366838736
epochs =  6 loss value =  3.7007805055203695
epochs =  8 loss value =  2.708548838741864
epochs =  10 loss value =  1.8646441471986142
epochs =  12 loss value =  1.2524609627607834
epochs =  14 loss value =  0.8730233612691779
epochs =  16 loss value =  0.6542578974583587
epochs =  18 loss value =  0.5267793196972503

Elapsed Time =>  0:32:04.116542


#### Accuracy 검증

In [6]:
test_input_data = test_data[ :, 0:-1 ]
test_target_data = test_data[ :, -1 ]

(true_list, false_list, index_label_prediction_list) = obj.accuracy(test_input_data, test_target_data)

Accuracy =>  0.7521806054386865


In [10]:
print(index_label_prediction_list)
print('\nfalse predicton number = ', len(index_label_prediction_list))

[[8, 1.0, 0], [9, 1.0, 0], [22, 1.0, 0], [30, 1.0, 0], [33, 1.0, 0], [34, 1.0, 0], [35, 1.0, 0], [43, 1.0, 0], [45, 1.0, 0], [58, 1.0, 0], [66, 1.0, 0], [70, 1.0, 0], [72, 1.0, 0], [75, 1.0, 0], [79, 1.0, 0], [85, 1.0, 0], [89, 1.0, 0], [90, 1.0, 0], [91, 1.0, 0], [95, 1.0, 0], [98, 1.0, 0], [114, 1.0, 0], [120, 1.0, 0], [128, 1.0, 0], [129, 1.0, 0], [134, 1.0, 0], [136, 1.0, 0], [138, 1.0, 0], [142, 1.0, 0], [143, 1.0, 0], [144, 1.0, 0], [146, 1.0, 0], [149, 1.0, 0], [150, 1.0, 0], [153, 1.0, 0], [160, 1.0, 0], [170, 1.0, 0], [172, 1.0, 0], [174, 1.0, 0], [175, 1.0, 0], [180, 1.0, 0], [188, 1.0, 0], [189, 1.0, 0], [190, 1.0, 0], [194, 1.0, 0], [196, 1.0, 0], [199, 1.0, 0], [205, 1.0, 0], [207, 1.0, 0], [208, 1.0, 0], [218, 1.0, 0], [219, 1.0, 0], [220, 1.0, 0], [226, 1.0, 0], [235, 1.0, 0], [243, 1.0, 0], [245, 1.0, 0], [249, 1.0, 0], [252, 1.0, 0], [258, 1.0, 0], [261, 1.0, 0], [264, 1.0, 0], [270, 1.0, 0], [271, 1.0, 0], [273, 1.0, 0], [274, 1.0, 0], [283, 1.0, 0], [289, 1.0, 0], [2